In [ ]:
import numpy as np
import pickle
import os
from cobaya.theory import Theory
from cobaya.run import run
from getdist.mcsamples import MCSamplesFromCobaya
from theory_polycamb import PolyCAMB


# ======================================================
# 2. 配置运行参数 (复制 planck_polycamb.yaml)
# ======================================================
config = {
    "output": "chains/planck_polycamb",  # 链和日志的文件夹
    # "suffix": "",
    "debug": True,                      # 初始运行启用调试
    
    # 自定义理论
    "theory": {
        "my_polycamb": {
            "external": PolyCAMB  # 直接使用上面定义的类
        }
    },
    "theory_name": "my_polycamb",
    
    # Planck 2018似然
    "likelihood": {
        # "planck_2018_lowl.TT": None,
        "planck_2018_highl_plik.TT": {
            "requires": {"Cl": {"TT": None}},
        }
    },
    
    # 宇宙学参数和先验
    "params": {
        "omega_b": {
            "prior": {"min": 0.01, "max": 0.024},
            "latex": r"\Omega_b h^2"
        },
        "omega_c": {
            "prior": {"min": 0.09, "max": 0.15},
            "latex": r"\Omega_c h^2"
        },
        "H0": {
            "prior": {"min": 55, "max": 80},
            "latex": r"H_0"
        },
        "logA": {  # ln(10^{10} A_s)
            "prior": {"min": 2.7, "max": 3.2},
            "latex": r"\ln(10^{10} A_s)"
        },
        "ns": {
            "prior": {"min": 0.88, "max": 1.02},
            "latex": r"n_s"
        },
        "tau": {
            "prior": {"min": 0.02, "max": 0.12},
            "latex": r"\tau"
        }
    },
    
    # MCMC采样器设置
    "sampler": {
        "mcmc": {
            "max_samples": 1000,     # 减少以加速测试
            "Rminus1_stop": 0.01,
            "covmat": "auto",
            "burn_in": 0.2,            # 调整burn-in
            "proposal_scale": 2.0      # 按需调整
        }
    },
    
    # 后处理
    # "post": {
    #     "suffix": "_test",
    #     "getdist": {
    #         "params": ["omega_b", "omega_c", "H0", "ns", "logA", "tau", "sigma8"],
    #         "contour": [0.68, 0.95]
    #     }
    # }
}



In [ ]:
os.makedirs("chains", exist_ok=True)
os.makedirs("emulators", exist_ok=True)

# 运行Cobaya
updated_info, sampler = run(config)

# 获取结果样本
samples = sampler.products()["sample"]

# 使用GetDist进行分析
gd_samples = MCSamplesFromCobaya(updated_info, samples.chain)

# 打印参数摘要
print("\nParameter summary:")
print(gd_samples.getTable(limit=1).table)

# 绘制三角形图
print("\nGenerating triangle plot...")
g = gd_samples.plot_2d(['omega_b', 'omega_c', 'H0', 'ns', 'logA', 'tau'], filled=True)
g.savefig("chains/planck_polycamb_triangle.pdf")

print("Analysis completed! Results saved in chains/ directory.")

 2025-06-28 01:37:34,009 [output] Output to be read-from/written-into folder 'chains', with prefix 'planck_polycamb'
